# Processing Pasadena (Texas) jail logs: 8/18 - 5/21

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
import altair as alt
import os

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Read the raw data parsed from 90-page PDF

In [4]:
pdf_data = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQFl6SwQ0V_c4hU0R8gpbnUAOQJyfezGdoFpzQabLVYmCpH8wD-oSmU9fCs3qKHXftaNdPp72I3t1sn/pub?output=csv",
    header=None,
    names=[
        "id",
        "date",
        "time",
        "shift",
        "reporting_personnel",
        "number_assisting",
        "location",
        "narrative",
    ],
    parse_dates=["date"],
    dtype={"id": "str", "time": str, "number_assisting": str},
)

In [5]:
pdf_data.sort_values("date", ascending=False).head()

,id,date,time,shift,reporting_personnel,number_assisting,location,narrative
606,202100126,2021-05-05,2240,NITE,"KENNEDY, J",3,DWI \nROOM,"Prisoner came into the jail intoxicated and was restrained in the chair due to the male needing a blood draw. [05/06/2021 01:08, JEKENNEDY, , PPD] Prisoner arrived to the DWI room and at removal of handcuffs by Ofc Green the inmate threw himself on to \nPAY \nthe ground stating he needs medical. Inmate was observed to have dry blood on his right hand and stated he was in pain and that he was diabetic. Inmate was then checked and further talked to in spanish by PSO Villegas. PSO Villegas advised that he was in pain due to his hand hurting and was given instructions to sit up and to allow for further search of property. Prisoner refused and continued to lay on ground. Officer Green advised that he was going to be a Magistrate Blood Draw and that restraint chair would be required for inmate.Inmate was restrained without further incident. [05/06/2021 01:13, JEKENNEDY, , PPD] \nPAY"
605,202100124,2021-04-30,100,NITE,"VILLEGAS, L",4,SALLY \nPORT,"Inmate was restrained in Sally Port at arrival due to being belligerent at arrest location. No further incident occurred. [04/30/2021 03:55, LVILLEGAS, , PPD] \nPA"
604,202100121,2021-04-27,2335,NITE,"VILLEGAS, L",2,BOOKING AREA,"INMATE ARRIVED BELLIGERENT AND STARTED PULLING FROM OFFICER HERNANDEZ AND MYSELF. PRISONER WAS INFORMED TO COOPERATE AND SHE CONTINUED TO PULL AND TRY TO STOP FROM ENTERING THE FACILITY. [04/28/2021 01:52, LVILLEGAS, , PPD] \nPA"
603,202100116,2021-04-25,1000,DAYS,"MARROQUIN, J",3,BOOKING AREA,"Officer Powell informed Medina,Christopher Booking No 258324 that he would be getting his blood drawn due to his \nDOB\nDWI charge. Medina stated he would not comply. Once Acadian arrived to the jail, Medina was placed in the restraint chair by Officer Jones, Officer Doskocil and myself. There was minimal resistance from Medina during the process. All straps were checked and he was safely secured in the restraint chair. During the blood draw, Medina began to make threats towards the arresting officer claiming he would beat him up once released from the restraint chair. Medina was also verbally abusive towards the medic that drew his blood. Medina remained in the restraint chair for the officer and jail staff safety. PSO Supervisor Davis was notified of the incident. [04/25/2021 13:16, JMARROQUIN, , PPD] \nPA"
602,202100114,2021-04-20,520,NITE,"BETANCOURT, L",3,CELL,"At approximately 0520 prisoner Gomez, Lance Giovanni Booking #258265 was behaving erratically and placed in \nDOB\nthe restraint chair after continously hitting the cell door, punching the phone and kicking the toilet on multiple times. Prisoner Gomez refused to stop and became belliegerent after being advised that jail staff would have to place him in a restraint chair for his own safety and to prevent damage to the facility. PSO Supervisor Adame attempted to deescalate the situation but the prisoner again stated that he would not calm down and wanted to be placed in to the restraint chair. Pso Burden, Pso Adame, and myself Pso Betancourt placed the prisoner in to the restraint chair with some resistance but no injuries to jail staff or the inmate. PSO Supervisor Burden was present on scene. [04/20/2021 05:50, LBETANCOURT, , PPD] \nPA"


### Six pages of the PDF were flat and have been parsed using a separate OCR process

In [6]:
flat_pages_data = pd.read_csv(
    "data/raw/restraint_slices_PDF/flat_pdfs_to_parse/parsed_data_from_flat_pdfs/all_data_from_parsed_flat_pages.csv",
    header=None,
    skiprows=1,
    names=[
        "id",
        "date",
        "time",
        "shift",
        "reporting_personnel",
        "number_assisting",
        "location",
        "narrative",
    ],
    parse_dates=["date"],
    dtype={"id": "str", "time": str, "number_assisting": str},
)

In [7]:
flat_pages_data.dropna(inplace=True)

In [8]:
flat_pages_data.head()

,id,date,time,shift,reporting_personnel,number_assisting,location,narrative
2,201800006,2018-05-14,25,NITE,"PINA, F",3,BOOKING AREA,"05/14/18 at 0023, Dispatch called to advise Jail Staff to have a restraint chair ready as a belligerent male was en route to the Jail. At 0025, Officer Logan arrived to the booking area with B/M Alexander, Devon DOB Book ID# 238901. Alexander was actively resisting, refusing to walk into the Jail on his own, and being carried by Officer Logan. Myself, PSO C. Weiss, and PSO K. Walker immediately assisted with placing FPINA, PA PPD] Alexander into the restraint chair. We were advised by Officer Logan that Alexander made 2 attempts to slip out of cuffs, and removed himself from his safety belt in the patrol vehicle during transport. After all straps were secured, Alexander was moved to Restraint Cell 1707 for observation. Approximately 1 hr. later, Officer Logan advised that he wished to speak with Alexander in order to make attempts to deescalate his behavior, and gain compliance so that the subject could be processed immediately. Myself and Officer Logan checked by Restraint Cell 1707, and Alexander advised he would be willing to do so. Just prior to being removed from all straps, I removed Alexander's locks to find a small, clear plastic bag containing a white crystalline substance. I immediately handed this substance to Officer Logan. Still willing to comply, myself and Officer Logan removed Alexander from the restraint chair, and guided him to ID/AFIS for processing. While assisting PSO K. Walker with prints, Alexander became increasingly agitated, and advised that he would refuse to answer all questions, and requested to be placed back into the restraint chair. After fingerprinting was completed, PSO K. Walker, PSO C. Weiss, Officer Logan, and myself escorted Alexander back to Restraint Cell 1707, and secured him into the chair for further observation. PSO Supervisor M. Walls notified. [05/14/2018 02:26, FPINA, PA PPD]"
7,201800009,2018-05-14,1600,EVES,"WALKER, K",3,SALLY PORT,"Inmate arrived at the jail verbally belligerent and was continuously banging his heade against the arresting officers unit causing his head to bleed. [05/14/2018 21:59, KAWALKER, PA , PPD]"
10,201800012,2018-05-19,2155,EVES,"HO, T",9,CELL,"At aproximately 2155 PSO Walker,K #6743 attempted to AFIS prisoner Wheatley,Jame- DOB The prisoner was being\nbelligerent so PSO Herrera#3260, Officer Maldonado,A 41 P and Officer Cambell ,J #0856 were on stand by in case there was any\nproblems. The prisoner began to resist jail staff from scanning his finger prints. Thats when PSO Herrera and officer Maldonado stepped in and put the prsioners arm behind his back and walked him to cell 1708 to be placed in the restraint chair. [05/19/2018\n22:57, THO, PA PPD1 [05/19/2018 23:00, THO, PA , PPD1"
17,201800015,2018-05-21,350,NITE,"PINA, F",4,SALLY PORT,"Occ'd at Sally Port 0350. Rest Chair requested at Sally Port as DWI Officer Maldonado advised subject was banging his head in patrol\nvehicle while en route to the Jail. [05/21/2018 04:33, FPINA, PA PPD]"
19,201800018,2018-05-22,2110,EVES,"RHODES, R",2,SALLY PORT,"""PRISONER HUDSON WHILE BEING TRANSPORTED TO THE JAIL WAS PURPOSEFULLY STRIKING HIS HEAD AGAINST THE\nCAGE INSIDE THE PATROL UNIT. HUDSON WAS PLACED INTO RESTRAINT CHAIR UPON ARRIVAL TO PREVENT HUDSON""\nFROM CONTINUING TO ATTEMPT TO SELF HARM [05/22/2018 21:18, RRHODES, PA , PPD]"


### Concatenate the two dataframes

In [9]:
src = pd.concat([pdf_data, flat_pages_data], ignore_index=True)

In [10]:
len(src)

648

### Dates

In [11]:
src["date"] = pd.to_datetime(src["date"], format="%m/%d/%Y")
src["date"] = pd.to_datetime(src["date"].dt.strftime("%Y-%m-%d"))
src["year"] = src["date"].dt.year
src["quarter"] = src["date"].dt.quarter
src["day"] = src["date"].dt.day
src["month"] = src["date"].dt.month
src["weekday"] = src["date"].dt.day_name()
src["monthname"] = src["date"].dt.month_name()
src["month_year"] = pd.to_datetime(src["date"]).dt.to_period("M")
src["month_year_full"] = src["date"].apply(lambda x: x.strftime("%B-%Y"))
src["month_year_full"] = pd.to_datetime(src["month_year_full"])

### Times

In [12]:
src["time"] = src["time"].astype(str).str.zfill(4)
src["hour"] = src["time"].str[:2]
src["minute"] = src["time"].str[-2:]
src["time_full"] = pd.to_datetime(
    src["hour"] + ":" + src["minute"], format="%H:%M"
).dt.time
src["hour"] = src["hour"].astype(int)
src["minute"] = src["minute"].astype(int)

In [13]:
src["location"] = src["location"].str.replace("\n", " ", regex=False)

### Make a copy of the dataframe and rearrange the columns

In [14]:
df = src[
    [
        "id",
        "date",
        "time",
        "shift",
        "reporting_personnel",
        "number_assisting",
        "location",
        "year",
        "quarter",
        "day",
        "month",
        "weekday",
        "monthname",
        "month_year",
        "month_year_full",
        "hour",
        "minute",
        "time_full",
        "narrative",
    ]
].copy()

### How many restraint cases do we have now? 

In [15]:
len(df)

648

---

### How many restraint cases per year? 

In [16]:
years = df.groupby("year")["id"].size().reset_index(name="count")
years

### What's the daily/weekly restraint incidents rate by year?

In [87]:
### How many days each year (some years are incomplete)
annual_days = []

for y in df.year.unique():
    annual_days.append(
        dict(
            year=y,
            days_num=(
                df[df["year"] == y]["date"].max() - df[df["year"] == y]["date"].min()
            ).days,
        )
    )
day_count = pd.DataFrame(annual_days)

In [88]:
rates = pd.merge(years, day_count, on="year")

In [91]:
rates["per_week"] = ((rates["count"] / rates["days_num"]) * 7).round(2)
rates["per_day"] = (rates["count"] / rates["days_num"]).round(2)

In [94]:
# would need to normalize this with jail intake figures or arrests, but they appear to be on the decline
rates

,year,count,days_num,per_week,per_day
0,2018,162,231,4.91,0.70
1,2019,252,362,4.87,0.70
2,2020,178,360,3.46,0.49
3,2021,56,123,3.19,0.46


### How many per month

In [17]:
months = df.groupby(["month", "year"])["id"].size().reset_index(name="count")
months.head()

,month,year,count
0,1,2019,20
1,1,2020,13
2,1,2021,12
3,2,2019,36
4,2,2020,11


In [18]:
chair_months = (
    alt.Chart(months)
    .mark_bar()
    .encode(
        alt.X("month:O", title="Month", axis=alt.Axis(grid=False)),
        y=alt.Y(
            "count:Q",
            title="Count",
            axis=alt.Axis(
                tickCount=6,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                title="Restrain incidents per month",
            ),
        ),
        facet=alt.Facet("year:O", columns=4),
    )
    .properties(
        width=300, height=300, title="Chair restraint frequency, by month and year"
    )
)
chair_months.configure_view(strokeOpacity=0)

alt.Chart(...)

### Which weekdays had the most incidents? 

In [20]:
weekdays = df.groupby("weekday").size().reset_index(name="count")
weekdays.sort_values("count", ascending=False)

,weekday,count
2,Saturday,118
3,Sunday,114
0,Friday,93
6,Wednesday,93
5,Tuesday,91
4,Thursday,76
1,Monday,63


### At which hours were these incidents most common? 

In [21]:
hours = df.groupby("hour").size().reset_index(name="count")
hours.sort_values("count", ascending=False)

,hour,count
2,2,47
3,3,47
1,1,47
21,21,46
0,0,42
17,17,39
4,4,37
14,14,34
20,20,34
19,19,33


### Chart the hours

In [22]:
chair_hours = (
    alt.Chart(hours)
    .mark_bar()
    .encode(
        alt.X("hour:O", title="Hour", axis=alt.Axis(grid=False)),
        y=alt.Y(
            "count:Q",
            title="Count",
            axis=alt.Axis(
                tickCount=6,
                gridColor="#dddddd",
                offset=6,
                tickSize=0,
                domainOpacity=0,
                title="Restraints per month",
            ),
        ),
    )
    .properties(
        width=600, height=330, title="Chair restraint frequency, by hour in day"
    )
)
chair_hours.configure_view(strokeOpacity=0)

alt.Chart(...)

### Which locations were most common? 

In [23]:
locations = df.groupby("location").size().reset_index(name="count")
locations.sort_values("count", ascending=False)

,location,count
3,CELL,233
2,BOOKING AREA,187
8,SALLY PORT,143
0,AFIS ROOM,28
9,SALLY PORT,13
4,DWI ROOM,9
6,JUVENILE HOLDING AREA,3
1,AFIS ROOM,2
5,DWI ROOM,1
7,LAUNDRY ROOM,1


### Who logged restraints the most (i.e., who to call)? 

In [24]:
personnel = df.groupby("reporting_personnel").size().reset_index(name="count")
personnel.sort_values("count", ascending=False).head(10)

,reporting_personnel,count
2,"BETANCOURT, L",94
12,"HO, T",58
11,"HERRERA, M",52
18,"PINA, F",50
15,"MARROQUIN, J",45
29,"WALKER, K",32
24,"THOMAS, R",31
0,"ADAME, R",27
6,"CORTES, A",25
17,"OROZCO, C",24


### Which shifts were most common? 

In [25]:
shifts = df.groupby("shift").size().reset_index(name="count")
shifts.sort_values("count", ascending=False).head(10)

,shift,count
2,NITE,316
1,EVES,274
0,DAYS,58


### Dates with the most incidents per day

In [56]:
per_day = df.groupby(["date"]).size().reset_index(name="count")
per_day.sort_values("count", ascending=False).head(5)

,date,count
431,2021-02-20,5
326,2020-05-24,4
336,2020-06-10,4
128,2019-02-08,4
195,2019-06-15,3


### What happened on Feb. 20, 2021? 

In [95]:
df[df["date"] == "2021-02-20"]

,id,date,time,shift,reporting_personnel,number_assisting,location,year,quarter,day,month,weekday,monthname,month_year,month_year_full,hour,minute,time_full,narrative,carter
571,202100048,2021-02-20,0100,NITE,"BOWMAN, D",3,BOOKING AREA,2021,1,20,2,Saturday,February,2021-02,2021-02-01,1,0,01:00:00,"Inmate Cruz,Melissa w/f booking# 257612 was belligerent and highly intoxicated on arrival, yelling at and insulting jail staff \nDOB\nand officers, as well as making sexual proposals to the jailer searching her. Inmate Cruz was then placed in the restraint chair at 0100 hrs after spitting through the cage at PSO Antos and Officer Inns, as well as later spitting on her statuatory warning paperwork. Sgt Buckert was notified. No injuries to jail staff or inmate Cruz during incident. [02/20/2021 03:28, DBOWMAN, , PPD] \nP",False
572,202100049,2021-02-20,0230,NITE,"BOWMAN, D",2,DWI ROOM,2021,1,20,2,Saturday,February,2021-02,2021-02-01,2,30,02:30:00,"inmate Vasquez,Benjamin w/m booking# 257608 was hihgly intox on arrival and was banging on the door multiple times to his holding cell after being told to stop several times. When brought to the DWI room inmate Vasquez refused a blood draw and was placed in the restraint chair in order to execute blood draw warrant. Inmate Vasquez then began rocking the chair back and forth trying to get loose. [02/20/2021 05:38, DBOWMAN, , PPD] \nP",False
573,202100047,2021-02-20,2330,NITE,"BOWMAN, D",3,AFIS ROOM,2021,1,20,2,Saturday,February,2021-02,2021-02-01,23,30,23:30:00,"Inmate Garcia,Manuel w/m booking# 257606 was belligerent upon arrival. While being AFISed inmate advised jail staff that \nDOB\nhe had a pre-existing injury to his right ring finger. While being AFISed inmate Garcia pulled away multiple times from jailer while doing fingerprints and was advised to stop. Inmate Garcia pulled away again and was put in handcuffs and secured into the restraint chair by PSOs Bowman and Betancourt along with Ofc Inns. Sgt Johnson was notified. No injuries to jail staff or inmate occured during incident. Inmate Garcia was placed in cell 1708 for further observation. [02/20/2021 00:23, DBOWMAN, , PPD] \nP",False
574,202100052,2021-02-20,0400,NITE,"BOWMAN, D",3,BOOKING AREA,2021,1,20,2,Saturday,February,2021-02,2021-02-01,4,0,04:00:00,"Inmate Enriquez,Ray w/m booking# 257614 was highly intoxicated when brought in and was verbally abusive towards \nDOB\nOfficers and jail staff making threats of physical violence. Inmate Medrano was attempting to resist orders from officers and attempted to kick Officer Torres while being searched. Inmate Enriquez was placed in the restraint chair at this time to avoid further acts of non compliance. Sgt Buckert notified. No injuries to staff or inmate occured during incident. [02/20/2021 05:55, DBOWMAN, , PPD] \nP",False
575,202100051,2021-02-20,0520,NITE,"BOWMAN, D",3,BOOKING AREA,2021,1,20,2,Saturday,February,2021-02,2021-02-01,5,20,05:20:00,"Inmate Medrano, Frank w/m booking# 257615 was Highbly beligerent to jail staff and officers, being verbally abusive and \nDOB\nespousing several racial slurs towards officers. Inmate Medrano refused to comply with commands when being checked by EMS crew and would not stay sitting. Inmate Medrano made multiple threats to jail staff and officers. Inmate Medrano was placed in the restraint chair by PSOs Betancourt and Bowman along with Officers Torres and Edwards. Inmate Medrano continued being verbally abusive once placed in the chair and made multiple threats to all parties present. Sgt Buckert was advised. No injuries to staff or inmate occured during incident. [02/20/2021 05:46, DBOWMAN, , PPD] \nP",False


---

### Just Carter cases

In [29]:
df["carter"] = df["narrative"].str.lower().str.contains("carter")

In [30]:
carter = df[df["carter"] == True]

In [31]:
carter.tail()

,id,date,time,shift,reporting_personnel,number_assisting,location,year,quarter,day,month,weekday,monthname,month_year,month_year_full,hour,minute,time_full,narrative,carter
403,202000040,2020-02-11,2205,NITE,"BETANCOURT, L",3,BOOKING AREA,2020,1,11,2,Tuesday,February,2020-02,2020-02-01,22,5,22:05:00,"At approximately 2210 prisoner Meehan,Brandon Lee was placed in the wrap after refusing to stand up and come out \nDOB\nof the holding cell. The prisoner became belligerent while also using profanity and pulling away when arresting Ofc Carter attempted to talk to the prisoner and assist the prisoner off the floor. I assisted Ofc Carter in getting the prisoner off the floor when the prisoner began pulling away from Ofc Carter. At this point Ofc Carter and my self placed the prisoner`s arms behind his back and requested the wrap system. We placed the prisoner in the wrap system with the help of PSO Walker and PSO Jalil. After being placed in cell 1708 Meehan began complaining of having high blood pressure problems, mental health problems such as bipolar disorder and being schizophrenic and then an hour and a half after being in the wrap the priosoner started to complain about having pains in his stomach due to a previous herina which he said he had already been having pains before coming to the jail. When i came back to the speak to the prisoner on a check shortly after the prisoner admitted that he just wanted to get out of jail and that he could sign a release for medical in order to get released. I explained to the prisoner that if he was seen by medical and needed to go to the hospital he would be released to go but would not be getting released to go home straight from the jail. When EMS did a vitals check on the prisoner they said everything looked normal and there was no need to worry especially about the blood pressure being too high. Meehan insisted in going because of pain in stomach due to the herina. Prisoner Meehan,Brandon was transported to the hospital and released from the jail with a rojo. [02/11/2020 01:38, LBETANCOURT, , PPD] \nPA",True
527,202000321,2020-11-11,1830,EVES,"MOODY, P",3,BOOKING AREA,2020,4,11,11,Wednesday,November,2020-11,2020-11-01,18,30,18:30:00,"At approximately 1830, inmate Jackson, Keenan was brought to jail by officer J. Carter. During the searching process \nDOB\ninmate began to not follow directions given by officer Carter and myself. Inmate then told jail staff if we took the handcuffs off to search him he would not co operate with jail staff orders and even attempt to fight jail staff. Officer Carter and myself attempted to reason with Mr. Jackson multiple times to co operate with our booking and searching procedures and to make his stay with us as easy as possible. Mr. Jackson continued to raise his voice at jail staff and promising that he would fight if he was removed from handcuffs. Jail staff then placed Mr. Jackson into the restraint chair with little resistance. No further conflict occurred. [11/11/2020 18:54, GGONZALEZ, , \nPAY \nPPD]",True
556,202100012,2021-01-19,1915,EVES,"OROZCO, C",2,SALLY PORT,2021,1,19,1,Tuesday,January,2021-01,2021-01-01,19,15,19:15:00,"Inmate was placed into restraint chair upon arrival by myself, PSO Cortes and Ofc Carter. Due to threatening to kill officers while en route to the jail and making threats to harm staff [01/19/2021 20:34, COROZCO, , PPD] \nPA",True
585,202100074,2021-03-15,1804,EVES,"KIDD, J",2,CELL,2021,1,15,3,Monday,March,2021-03,2021-03-01,18,4,18:04:00,"Inmate pushed button notifying jailers that he was suicidal. Inmate was then placed into the restraint chair by PSO Kidd and PSO Teel. Inmate was put into chair with no further incident and Sgt Hibbs was notified. [03/15/2021 18:05, JKIDD, , PPD] After inmate was \nPAY \nin chair for approximately 10 minutes PSO Teel made the decision to move the inmate to the wrap. The inmate was then moved to the wrap by PSO Kidd, PSO Teel, and OFC Carter. [03/15/2021 18:23, JKIDD, , PPD] Inmate was move

---

### Exports

In [32]:
carter.to_csv("data/processed/carter_restraint_cases.csv", index=False)

In [33]:
df.to_csv("data/processed/all_restraint_cases.csv", index=False)

In [34]:
chair_hours.save("visuals/chair_hours_frequency.png", scale_factor=1)

In [35]:
chair_months.save("visuals/chair_month_year_frequency.png", scale_factor=1)